# Лабораторная 5: Проверка гипотез 2

## Задание 1: сравнение критерия знаков и критерия ранговых знаков (Вилкинсона)


### Пункт 1: критерий знаков

Будем рассматривать выборку $X: x_j \in N(0, 7), n = 20$ и сравнивать ее с
а) $Y: y_j = 5x_j + \epsilon_j, \epsilon_j \in Uniform[-10, 10]$
б) $Y: y_j = 5x_j + \eta_j, \eta_j \in expon(20)$

In [ ]:
import numpy as np

distr1 = np.random.normal(loc=5, scale=7 ** 0.5, size=200)